In [42]:
# Import necessary libraries
import os
import subprocess
import json

# Imposta la variabile d'ambiente PRIMA di aprire il video
os.environ["OPENCV_FFMPEG_READ_ATTEMPTS"] = "100000"  # Aumenta ulteriormente se necessario

import cv2
import random
from ultralytics import YOLO

# --- CARICAMENTO MODELLO YOLO ---
confidence_threshold = 0.6
best_model = YOLO("/home/lorenzo/Scaricati/best.pt")

# --- CONFIGURAZIONE PATH ---
file_name = "GH010213-2.mp4"
input_video_path = "/home/lorenzo/Documenti/GitHub/Computer-Science-Sapienza/Computer Vision/project/test_video/" + file_name          # video in 1920x1080
temp_scaled_video_path = "/home/lorenzo/Documenti/GitHub/Computer-Science-Sapienza/Computer Vision/project/test_video/" + "temp_scaled_" + file_name      # file temporaneo in 640x384
output_video_path = "/home/lorenzo/Documenti/GitHub/Computer-Science-Sapienza/Computer Vision/project/test_video/output/out_conf_" + str(int(confidence_threshold * 100)) + "_" + file_name          # video finale in 1920x1080

In [43]:
# --- STEP 1: CREA VIDEO TEMPORANEO SCALATO A 640x384 ---
cap_in = cv2.VideoCapture(input_video_path)
if not cap_in.isOpened():
    raise Exception("Impossibile aprire il video di input.")

# Ottieni informazioni dal video originale
orig_width  = int(cap_in.get(cv2.CAP_PROP_FRAME_WIDTH))   # 1920
orig_height = int(cap_in.get(cv2.CAP_PROP_FRAME_HEIGHT))  # 1080
fps         = cap_in.get(cv2.CAP_PROP_FPS)
total_frames = int(cap_in.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Video di input: {input_video_path}")
print(f"Dimensione: {orig_width}x{orig_height}")
print(f"FPS: {fps}")
print(f"Numero di frame: {total_frames}")

# Dimensione di training di YOLO
yolo_width, yolo_height = 640, 384

# Scrittore per il video temporaneo (usa codec 'mp4v' per il file intermedio)
fourcc_temp = cv2.VideoWriter_fourcc(*'mp4v')
temp_writer = cv2.VideoWriter(temp_scaled_video_path, fourcc_temp, fps, (yolo_width, yolo_height))

frame_idx = 0
while True:
    ret, frame = cap_in.read()
    if not ret:
        break
    # Ridimensiona il frame da 1920x1080 a 640x384 (non mantiene aspect ratio originale)
    frame_resized = cv2.resize(frame, (yolo_width, yolo_height))
    temp_writer.write(frame_resized)
    frame_idx += 1

cap_in.release()
temp_writer.release()
print("Video temporaneo scalato creato.")

Video di input: /home/lorenzo/Documenti/GitHub/Computer-Science-Sapienza/Computer Vision/project/test_video/GH010213-2.mp4
Dimensione: 1920x1080
FPS: 29.97002997002997
Numero di frame: 8064
Video temporaneo scalato creato.


In [44]:
# ----- STEP 2: ESEGUI PREDIZIONI CON YOLO SUL VIDEO TEMPORANEO -----
# Calcola lo skip per ottenere circa 15 fps di predizione
skip_factor = max(1, round(fps / 15))
print(f"Input fps: {fps:.2f}. Utilizzo vid_stride={skip_factor} per predire ~15 fps.")

# Utilizziamo un dizionario per salvare i risultati
# Chiave: indice del frame (nella scala 640x384) per cui è stata eseguita la predizione
# Valore: lista di dizionari contenenti: 'xywh', 'cls' e 'conf'
processed_results = {}

# Con il parametro vid_stride, YOLO processa solo i frame multipli di skip_factor
for i, result in enumerate(best_model.predict(source=temp_scaled_video_path, save=False, stream=True, vid_stride=skip_factor, conf=confidence_threshold)):
    processed_frame_idx = i * skip_factor  # indice del frame elaborato nella scala temporanea
    if result.boxes is not None and result.boxes.xywh is not None:
        xywh = result.boxes.xywh.cpu().numpy()    # coordinate in formato (x_center, y_center, width, height)
        cls = result.boxes.cls.cpu().numpy()        # indice della classe per ogni box
        conf = result.boxes.conf.cpu().numpy()      # confidenza per ogni box
        boxes_data = []
        for j in range(len(xywh)):
            boxes_data.append({
                'xywh': xywh[j],
                'cls': int(cls[j]),
                'conf': float(conf[j])
            })
    else:
        boxes_data = None
    processed_results[processed_frame_idx] = boxes_data

print("Predizioni eseguite sul video scalato.")

# Propaga le predizioni ai frame non processati: per ogni frame (0 .. total_frames-1)
# usiamo le bounding box del frame processato più vicino (precedente)
boxes_per_frame = []
processed_indices = sorted(processed_results.keys())
current_pointer = 0
last_boxes = processed_results[processed_indices[0]]  # in genere il frame 0

for i in range(total_frames):
    if current_pointer < len(processed_indices) and i == processed_indices[current_pointer]:
        last_boxes = processed_results[processed_indices[current_pointer]]
        current_pointer += 1
    boxes_per_frame.append(last_boxes)

Input fps: 29.97. Utilizzo vid_stride=2 per predire ~15 fps.

video 1/1 (frame 1/4032) /home/lorenzo/Documenti/GitHub/Computer-Science-Sapienza/Computer Vision/project/test_video/temp_scaled_GH010213-2.mp4: 384x640 1 crack, 94.0ms
video 1/1 (frame 2/4032) /home/lorenzo/Documenti/GitHub/Computer-Science-Sapienza/Computer Vision/project/test_video/temp_scaled_GH010213-2.mp4: 384x640 (no detections), 82.1ms
video 1/1 (frame 3/4032) /home/lorenzo/Documenti/GitHub/Computer-Science-Sapienza/Computer Vision/project/test_video/temp_scaled_GH010213-2.mp4: 384x640 (no detections), 93.5ms
video 1/1 (frame 4/4032) /home/lorenzo/Documenti/GitHub/Computer-Science-Sapienza/Computer Vision/project/test_video/temp_scaled_GH010213-2.mp4: 384x640 (no detections), 110.7ms
video 1/1 (frame 5/4032) /home/lorenzo/Documenti/GitHub/Computer-Science-Sapienza/Computer Vision/project/test_video/temp_scaled_GH010213-2.mp4: 384x640 (no detections), 467.0ms
video 1/1 (frame 6/4032) /home/lorenzo/Documenti/GitHub/Com

In [45]:
# ----- STEP 3: CREA VIDEO DI OUTPUT CON LE BB MAPPATE SU FRAME ORIGINALI -----
cap_orig = cv2.VideoCapture(input_video_path)
if not cap_orig.isOpened():
    raise Exception("Impossibile riaprire il video di input.")

# Qui uso 'mp4v' per scrivere il video intermedio; non possiamo impostare il bitrate con VideoWriter.
fourcc_out = cv2.VideoWriter_fourcc(*'mp4v')
orig_width  = int(cap_orig.get(cv2.CAP_PROP_FRAME_WIDTH))   # es. 1920
orig_height = int(cap_orig.get(cv2.CAP_PROP_FRAME_HEIGHT))   # es. 1080
fps         = cap_orig.get(cv2.CAP_PROP_FPS)
out_writer  = cv2.VideoWriter(temp_scaled_video_path, fourcc_out, fps, (orig_width, orig_height))

# Fattori di scala per mappare le coordinate da 640x384 a 1920x1080
scale_x = orig_width / yolo_width
scale_y = orig_height / yolo_height

# Mappa colori per le classi, basata su best_model.names
model_names = best_model.names  # es. {0: "person", 1: "car", ...}
random.seed(42)
colors = {cls: (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
          for cls in model_names.keys()}

frame_idx = 0
while True:
    ret, frame = cap_orig.read()
    if not ret:
        break
    # Recupera le BB per il frame corrente (nella scala 640x384)
    boxes = boxes_per_frame[frame_idx]
    if boxes is not None:
        for box in boxes:
            # Estrai coordinate, classe e confidenza
            x_center, y_center, w, h = box['xywh']
            class_id = box['cls']
            confidence = box['conf']
            # Scala le coordinate al formato originale
            x_center *= scale_x
            y_center *= scale_y
            w *= scale_x
            h *= scale_y
            top_left = (int(x_center - w/2), int(y_center - h/2))
            bottom_right = (int(x_center + w/2), int(y_center + h/2))
            # Seleziona il colore per la classe
            color = colors.get(class_id, (0, 255, 0))
            # Disegna la bounding box (più spessa)
            cv2.rectangle(frame, top_left, bottom_right, color, 3)
            
            # Crea l'etichetta con nome classe e confidenza in percentuale
            class_name = model_names.get(class_id, "Unknown")
            label = f"{class_name} {confidence * 100:.1f}%"
            
            # Parametri per il testo
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.8
            text_thickness = 2
            (text_width, text_height), baseline = cv2.getTextSize(label, font, font_scale, text_thickness)
            
            # Posiziona il rettangolo di background per il testo
            text_offset_x = top_left[0]
            text_offset_y = top_left[1] - 10  # 10 pixel sopra la BB
            if text_offset_y - text_height - baseline < 0:
                text_offset_y = top_left[1] + text_height + baseline + 10
            box_coords = ((text_offset_x, text_offset_y - text_height - baseline - 4),
                          (text_offset_x + text_width + 4, text_offset_y))
            # Disegna il rettangolo riempito (sfondo per il testo) con lo stesso colore della BB
            cv2.rectangle(frame, box_coords[0], box_coords[1], color, cv2.FILLED)
            # Disegna il testo in bianco sopra il rettangolo
            cv2.putText(frame, label, (text_offset_x + 2, text_offset_y - 2), 
                        font, font_scale, (255, 255, 255), text_thickness, cv2.LINE_AA)
    out_writer.write(frame)
    frame_idx += 1

cap_orig.release()
out_writer.release()
print(f"Video intermedio salvato come {temp_scaled_video_path}")

Video intermedio salvato come /home/lorenzo/Documenti/GitHub/Computer-Science-Sapienza/Computer Vision/project/test_video/temp_scaled_GH010213-2.mp4


In [46]:
# ----- STEP 4 (opzionale): MANTIENI LO STESSO BITRATE RI-ENCODANDO IL VIDEO DI OUTPUT -----

DO_IT = True

if DO_IT:
    def get_input_bitrate(video_path):
        """Estrae il bitrate del video di input utilizzando ffprobe."""
        cmd = [
            'ffprobe', '-v', 'error', '-select_streams', 'v:0',
            '-show_entries', 'stream=bit_rate', '-of', 'json', video_path
        ]
        output = subprocess.check_output(cmd)
        data = json.loads(output)
        bitrate = int(data['streams'][0]['bit_rate'])
        return bitrate

    # Estrae il bitrate dal video di input
    input_bitrate = get_input_bitrate(input_video_path)
    print(f"Bitrate video di input: {input_bitrate} bit/s")

    # Ri-encoda il video intermedio usando FFmpeg con il bitrate estratto
    ffmpeg_cmd = [
        'ffmpeg', '-y', '-i', temp_scaled_video_path,
        '-c:v', 'libx264', '-b:v', str(input_bitrate),
        output_video_path
    ]
    print("Esecuzione di FFmpeg per ri-encodifica...")
    subprocess.run(ffmpeg_cmd, check=True)
    print(f"Video finale salvato come {output_video_path}")
else:
    print("Non viene effettuata la ri-encodifica.")
    os.rename(temp_scaled_video_path, output_video_path)
    print(f"Video finale salvato come {output_video_path}")

Bitrate video di input: 28024669 bit/s
Esecuzione di FFmpeg per ri-encodifica...


ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.3.0 (conda-forge gcc 13.3.0-1)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1727723455708/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1727723455708/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1727723455708/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1727723455708/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1727723455708/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --enable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopen

Video finale salvato come /home/lorenzo/Documenti/GitHub/Computer-Science-Sapienza/Computer Vision/project/test_video/output/out_conf_60_GH010213-2.mp4


[out#0/mp4 @ 0x61ec6cead400] video:900058KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.009915%
frame= 8064 fps= 19 q=-1.0 Lsize=  900147KiB time=00:04:29.00 bitrate=27412.4kbits/s speed=0.641x    
[libx264 @ 0x61ec6cea71c0] frame I:36    Avg QP:13.66  size:281480
[libx264 @ 0x61ec6cea71c0] frame P:3971  Avg QP:16.53  size:140879
[libx264 @ 0x61ec6cea71c0] frame B:4057  Avg QP:19.47  size: 86787
[libx264 @ 0x61ec6cea71c0] consecutive B-frames: 30.4%  5.0%  8.1% 56.5%
[libx264 @ 0x61ec6cea71c0] mb I  I16..4: 17.9% 65.8% 16.4%
[libx264 @ 0x61ec6cea71c0] mb P  I16..4:  4.2% 19.0%  3.2%  P16..4: 30.6% 20.8% 11.6%  0.0%  0.0%    skip:10.6%
[libx264 @ 0x61ec6cea71c0] mb B  I16..4:  1.2%  7.8%  2.6%  B16..8: 40.5% 17.8%  5.0%  direct: 7.9%  skip:17.3%  L0:43.8% L1:24.3% BI:31.9%
[libx264 @ 0x61ec6cea71c0] final ratefactor: 15.73
[libx264 @ 0x61ec6cea71c0] 8x8 transform intra:70.5% inter:71.9%
[libx264 @ 0x61ec6cea71c0] coded y,uvDC,uvAC intra: 67.5% 66.

In [47]:
# --- Rimuovi il file temporaneo ---
try:
    os.remove(temp_scaled_video_path)
    print("File temporaneo rimosso.")
except Exception as e:
    print(f"Impossibile rimuovere il file temporaneo: {e}")

File temporaneo rimosso.
